## Install Spark

In [3]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-3.2.3-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://archive

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Vine-Review-Analysis').getOrCreate()

## Extract the dataset as a csv file


In [8]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [19]:
# Import the csv file as a DataFrame
vine_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load('/content/vine_table.csv')

In [20]:
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R3EQSTM9PWRAL|          3|            0|          0|   N|                Y|
| RBWPRK17XKIXD|          5|            0|          0|   N|                Y|
| RRSLOAF273XFC|          5|            1|          2|   N|                Y|
|R3S8W9Q6SWIT8O|          4|            0|          0|   N|                Y|
|R3QQ6NSLRVBFJC|          4|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



## Part 1 - Retrieve rows that have total votes that are 20 or greater

In [30]:
df_filtered_total_votes = vine_df.filter(vine_df.total_votes >= 20).sort(vine_df.total_votes.asc())

df_filtered_total_votes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1QHNDT8EEYGT2|          5|           19|         20|   N|                N|
|R1UN7MR4HXB5OB|          5|           19|         20|   N|                Y|
|R36FG0N8AGH2BL|          1|            6|         20|   N|                Y|
|R1DB6355KE79V4|          2|            2|         20|   N|                Y|
|R2C9YSW5FZUI55|          5|           13|         20|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



## Part 2 - Retrieve rows that have a percentage of helpful votes that is 50% or greater

In [35]:
from pyspark.sql.functions import col 

# Create the new column and calculate the percentage
df_helpful_votes = df_filtered_total_votes.withColumn("percentage_helpful_votes", col("helpful_votes") / col("total_votes"))

df_helpful_votes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R1QHNDT8EEYGT2|          5|           19|         20|   N|                N|                    0.95|
|R1UN7MR4HXB5OB|          5|           19|         20|   N|                Y|                    0.95|
|R36FG0N8AGH2BL|          1|            6|         20|   N|                Y|                     0.3|
|R1DB6355KE79V4|          2|            2|         20|   N|                Y|                     0.1|
|R2C9YSW5FZUI55|          5|           13|         20|   N|                N|                    0.65|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
only showing top 5 rows



In [36]:
# Filter for the rows where % of helpful votes is greater than or equal to 50% 
df_filtered_helpful_votes = df_helpful_votes.filter(df_helpful_votes.percentage_helpful_votes >= 0.5).sort(df_helpful_votes.percentage_helpful_votes.asc())

df_filtered_helpful_votes.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R3JLKND0U4YT6E|          1|           15|         30|   N|                Y|                     0.5|
| RCUZEDUM61BTF|          1|           10|         20|   N|                Y|                     0.5|
|R3SQVCMOKJBAPC|          5|           11|         22|   N|                Y|                     0.5|
|R1JDS7GVSDMLVL|          1|           16|         32|   N|                Y|                     0.5|
| REVRH4SZSXR56|          1|           12|         24|   N|                Y|                     0.5|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
only showing top 5 rows



## Part 3 - Retrieve rows where a review was written as part of the Vine program (paid)



In [40]:
# Filter for the rows where Vine is equal to 'Y' or yes and sort by ascending order as a check 
vine_paid_reviews = df_filtered_helpful_votes.filter(df_filtered_helpful_votes.vine == 'Y').sort(df_helpful_votes.vine.asc())

vine_paid_reviews.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R3VC6XDKPX0BBI|          4|           71|         82|   Y|                N|      0.8658536585365854|
|R10VXTKK26IPXN|          5|           98|        102|   Y|                N|      0.9607843137254902|
|R2Z01BR4BYGQTI|          4|           20|         26|   Y|                N|      0.7692307692307693|
|R2V7LXZUJTPLJL|          1|           21|         27|   Y|                N|      0.7777777777777778|
| RSZ9IDZ9Y3700|          4|          142|        154|   Y|                N|       0.922077922077922|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
only showing top 5 rows



## Part 4 - Retrieve rows where a review wasn't written as part of the Vine program (unpaid)

In [41]:
# Filter for rows where Vine is equal to 'N' or no and sort by descending order as a check 
unpaid_reviews = df_filtered_helpful_votes.filter(df_filtered_helpful_votes.vine == 'N').sort(df_helpful_votes.vine.desc())

unpaid_reviews.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|percentage_helpful_votes|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
|R3FWCZACX088J5|          1|           38|         66|   N|                Y|      0.5757575757575758|
|R1UI2GK0ZD6IQM|          1|           20|         38|   N|                Y|      0.5263157894736842|
|R1LUO5GU4SFH49|          3|           23|         27|   N|                N|      0.8518518518518519|
| RG7406AJVBUBK|          5|          157|        184|   N|                N|      0.8532608695652174|
|R1JX2GVHX1QYIN|          5|          158|        160|   N|                Y|                  0.9875|
+--------------+-----------+-------------+-----------+----+-----------------+------------------------+
only showing top 5 rows

